In [1]:
import logging
from winterapi import WinterAPI
from wintertoo.models import WinterRaDecToO

In [2]:
logging.getLogger("winterapi").setLevel("DEBUG")

In [3]:
winter = WinterAPI()

In [4]:
# Program details
program = "2024A001"

In [5]:
# Target details
ra_deg = 249.1311789
dec_deg = 47.858962
# Target name is a useful field for querying data, so put something useful!
# If you use Skyportal, we strongly advise you to use that name
target_name = "ZTF23aaxeacr"

In [6]:
# A ToO with the same dither combo for Y/J
too_1 = WinterRaDecToO(ra_deg=ra_deg, dec_deg=dec_deg, n_dither=8, t_exp=960., filters=['Y', 'J'], target_name=target_name)
too_1

WinterRaDecToO(ra_deg=249.1311789, dec_deg=47.858962, use_field_grid=False, filters=['Y', 'J'], target_priority=50.0, target_name='ZTF23aaxeacr', t_exp=960.0, n_exp=1, n_dither=8, dither_distance=30.0, start_time_mjd=60346.16884958786, end_time_mjd=60347.15884959191, max_airmass=2.0)

In [7]:
# A second ToO, with a different dither number for Hs
too_2 = WinterRaDecToO(ra_deg=ra_deg, dec_deg=dec_deg, n_dither=15, t_exp=960., filters=["Hs"], target_name=target_name)
too_2

WinterRaDecToO(ra_deg=249.1311789, dec_deg=47.858962, use_field_grid=False, filters=['Hs'], target_priority=50.0, target_name='ZTF23aaxeacr', t_exp=960.0, n_exp=1, n_dither=15, dither_distance=30.0, start_time_mjd=60346.16884958786, end_time_mjd=60347.15884959191, max_airmass=2.0)

In [8]:
too_list = [too_1, too_2]

In [9]:
program_list = winter.get_programs()
print(f"Available programs: {program_list}")
assert program in program_list, f"program {program} not found! Add this program first."

Available programs: ['2024A001', '2023A000', '2023A001', '2023A002']


In [10]:
# Schedule has three observations in total
local_schedule = winter.build_schedule_locally(
    program_name=program,
    data=too_list
)
local_schedule

,targName,raDeg,decDeg,fieldID,filter,visitExpTime,priority,progPI,progName,progID,validStart,validStop,observed,maxAirmass,ditherNumber,ditherStepSize,obsHistID
0,ZTF23aaxeacr,249.131179,47.858962,999999999,Y,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,8,30.0,0
1,ZTF23aaxeacr,249.131179,47.858962,999999999,J,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,8,30.0,1
2,ZTF23aaxeacr,249.131179,47.858962,999999999,Hs,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,15,30.0,2


In [11]:
# If submit_trigger is False, the API will just check the schedule but not put in a ToO
# Set this to True to actually trigger!
api_res, api_schedule = winter.submit_too(
    program_name=program,
    data=too_list,
    submit_trigger=False
)

In [12]:
# Let's see what message the API sent back
api_res.json()["msg"]

'Request passed validation! Triggering was set to False. Schedule was not saved to disk and has no assigned name.'

In [13]:
# Uncomment to trigger for real
api_res, api_schedule = winter.submit_too(
    program_name=program,
    data=too_list,
    submit_trigger=True
)
api_res.json()["msg"]

"Request passed validation! Triggering was set to True. Schedule name is 'request_2024A001_2024_02_05_19_49_26' ."

In [14]:
# This is the schedule sent back by the API. Reassuringly, it matches the local one
api_schedule

,targName,raDeg,decDeg,fieldID,filter,visitExpTime,priority,progPI,progName,progID,validStart,validStop,observed,maxAirmass,ditherNumber,ditherStepSize,obsHistID
0,ZTF23aaxeacr,249.131179,47.858962,999999999,Y,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,8,30.0,0
1,ZTF23aaxeacr,249.131179,47.858962,999999999,J,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,8,30.0,1
2,ZTF23aaxeacr,249.131179,47.858962,999999999,Hs,960.0,50.0,RStein,2024A001,3,60346.16885,60347.15885,False,2.0,15,30.0,2
